In [203]:
import os
os.environ['QT_API'] = 'pyqt'
os.environ['ETS_TOOLKIT'] = 'qt4'
#os.environ['SUBJECTS_DIR'] = "/path/to/subjects"

import sip
sip.setapi('QDate',2)

import os
import os.path as op
import numpy as np
from numpy import arange
from numpy.random import permutation
import nibabel as nib
from surfer import Brain

print(__doc__)


%gui qt


save output



In [204]:
subject_id = "fsaverage"
subjects_dir = os.environ["SUBJECTS_DIR"]

"""
Set number of datasets (studies, conditions, any other grouping for individual coordinate spheres)
"""
ndatasets = 8

In [205]:
"""
Create generator of unique RGB colors for plotting objects (such as coordinate spheres)
"""

import math

v = 1.0
s = 1.0
p = 0.0
def rgbcolor(h, f):
    """Convert a color specified by h-value and f-value to an RGB
    three-tuple."""
    # q = 1 - f
    # t = f
    if h == 0:
        return v, f, p
    elif h == 1:
        return 1 - f, v, p
    elif h == 2:
        return p, v, f
    elif h == 3:
        return p, 1 - f, v
    elif h == 4:
        return f, p, v
    elif h == 5:
        return v, p, 1 - f

def uniquecolors(n):
    """Compute a list of distinct colors, ecah of which is
    represented as an RGB three-tuple"""
    hues = [360.0 / n * i for i in range(n)]
    hs = (math.floor(hue / 60) % 6 for hue in hues)
    fs = (hue / 60 - math.floor(hue / 60) for hue in hues)
    return [rgbcolor(h, f) for h, f in zip(hs, fs)]

In [206]:
"""
Generate sphere colors
"""
datacolors=uniquecolors(ndatasets)
#datacolors[3]

In [207]:
"""
First we'll get a set of stereotaxic foci in the MNI
coordinate system. These might be peak activations from
a volume based analysis.
"""
coords_rh = [[26, 57, -8],
            [28, 38, 31],
            [28, 55, -1],
            [30, 57, -5],
            [36,63,6],
            [27,62,22],
            [9, 59, 34],
            [26,64,10],
            [30,52,8],
            [30,48,8],
            [20,68,10],
            [32,56,0]
                      ]
#study #1 = 0
coords_rh_studkey = [1,1,1,1,2,4,4,5,6,6,7,7]

coords_lh = [[-10, 66, 8],
         [-18,57,0],
         [-18,58,-8],
         [-24,62,22],
         [-9,59,28],
         [-21,56,-8],
         [-34,52,-4]
                      ]

#study #1 = 0
coords_lh_studkey = [0,2,3,4,4,4,5]

print(coords_lh_studkey)

[0, 2, 3, 4, 4, 4, 5]


In [208]:
"""
First we'll get a set of stereotaxic foci in the MNI
coordinate system. These might be peak activations from
a volume based analysis.
"""
# coords_rh_s2 = [[26, 57, -8],
#           [28, 38, 31],
#           [28, 55, -1],
#          [30, 57, -5]
        
#                       ]

# coords_rh_s3 = [
#          [36,63,6]
#                       ]
# coords_rh_s5 = [
#              [27,62,22],
#              [9, 59, 34]
#                       ]


# coords_rh_s6 = [
#     [26,64,10]
#                       ]


# coords_rh_s7 = [
#              [30,52,8],
#              [30,48,8]
#                       ]


# coords_rh_s8 = [
#              [20,68,10],
#              [32,56,0]
#                       ]


# coords_lh_s1 = [[-10, 66, 8]
#                       ]


# coords_lh_s3 = [
#           [-18,57,0]
#                                 ]

# coords_lh_s4 = [
#           [-18,58,-8]
#                       ]

# coords_lh_s5 = [
#          [-24,62,22],
#          [-9,59,28],
#              [-21,56,-8]
#                       ]

# coords_lh_s6 = [
#              [-34,52,-4]
#                       ]

"\nFirst we'll get a set of stereotaxic foci in the MNI\ncoordinate system. These might be peak activations from\na volume based analysis.\n"

In [209]:
"""

Add Yeo reference labels
"""

hemi = "lh"

"""
Read in the Buckner resting state network annotation. (This requires a
relatively recent version of Freesurfer, or it can be downloaded separately).
"""
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi + ".Yeo2011_17Networks_N1000.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)


"""
Make a random vector of scalar data corresponding to a value for each region in
the parcellation.

"""
rs = np.random.RandomState(4)
roi_data = rs.uniform(.5, .75, size=len(names))

"""
Make a vector containing the data point at each vertex.
"""
vtx_data_lh = roi_data[labels]


"""
repeat for rh
"""
hemi = "rh"
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi + ".Yeo2011_17Networks_N1000.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

vtx_data_rh = roi_data[labels]

## Deal with Left Hemisphere Data

In [210]:
currhem = "lh"

"""
Bring up the visualization.
"""
brain_lh = Brain(subject_id, currhem, "inflated")

#brain = Brain("fsaverage", "split", "inflated",
#              views=['frontal'], background="white")

In [211]:
"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.
"""
#brain.add_data(vtx_data_lh, .5, .75, colormap="GnBu", alpha=.9, hemi="lh")
#brain.add_data(vtx_data_rh, .5, .75, colormap="GnBu", alpha=.9, hemi="rh")

brain_lh.add_data(vtx_data_lh, .5, .75, colormap="GnBu", alpha=.9, hemi="lh")
#brain_lh.close()


In [212]:
"""
Now we plot the foci on the inflated surface. We will map
the foci onto the surface by finding the vertex on the "white"
mesh that is closest to the coordinate of each point we want
to display.

While this is not a perfect transformation, it can give you
some idea of where peaks from a volume-based analysis would
be located on the surface.

You can use any valid matplotlib color for the foci; the
default is white.
"""
#brain.add_foci(coords_rh, map_surface="white", color="green", scale_factor = 0.75, hemi="rh")

#brain.add_foci(coords_lh, map_surface="white", color="green", scale_factor = 0.75, hemi="lh")


'\nNow we plot the foci on the inflated surface. We will map\nthe foci onto the surface by finding the vertex on the "white"\nmesh that is closest to the coordinate of each point we want\nto display.\n\nWhile this is not a perfect transformation, it can give you\nsome idea of where peaks from a volume-based analysis would\nbe located on the surface.\n\nYou can use any valid matplotlib color for the foci; the\ndefault is white.\n'

In [213]:
"""
Map foci to this hemisphere.

This works through iterating through a list of datapoints, such that data from the same grouping gets \\
the same color code
"""
#for i in range(4,6):
#    subcoords = eval("coords_lh_s" + str(i))
#    print(subcoords)
#    brain_lh.add_foci(subcoords, map_surface="white", color=datacolors[i], scale_factor = 0.5, hemi="lh")    


#for i in coords_lh:
for i in range(len(coords_lh)):
    #subcoords = eval("coords_lh_s" + str(i))
    #print(i)
    print(coords_lh[i])
    print(coords_lh_studkey[i])
    brain_lh.add_foci(coords_lh[i], map_surface="white", color=datacolors[coords_lh_studkey[i]], scale_factor = 0.5, hemi="lh")    
  
#brain_lh.add_foci(coords_lh_s1, map_surface="white", color="b", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s3, map_surface="white", color="r", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s4, map_surface="white", color="c", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s5, map_surface="white", color="m", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s6, map_surface="white", color="y", scale_factor = 0.5, hemi="lh")


[-10, 66, 8]
0
[-18, 57, 0]
2
[-18, 58, -8]
3
[-24, 62, 22]
4
[-9, 59, 28]
4
[-21, 56, -8]
4
[-34, 52, -4]
5


In [214]:
"""
Various view and saving options do not play well with 'split' brain, and are best dealt with on a hemisphere basis
"""

#brain_rh.show_view(dict(azimuth=-42, elevation=105, distance=225,
#                     focalpoint=[-30, -20, 15]))

brain_lh.show_view('frontal')

((119.99999999999999,
  80.0,
  435.50091552734381,
  array([ 0.        ,  0.84741974,  0.        ])),
 106.739)

In [215]:
"""
save output
"""
#with multiple arguments, one can use a tuple as operand: 'foo %d, bar %d' % (foo, bar)
brain_lh.save_image("%s_lat_%s.png" % (subject_id, currhem))

## Deal with Right Hemisphere Data

In [216]:
currhem = "rh"

"""
Bring up the visualization.
"""
brain_rh = Brain(subject_id, currhem, "inflated")

In [217]:
"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.
"""
brain_rh.add_data(vtx_data_rh, .5, .75, colormap="GnBu", alpha=.9, hemi="rh")


In [218]:
#print(len(coords_rh))
#print(len(coords_rh_studkey))
#print(coords_rh[11])
#print(range(len(coords_rh)))

for i in range(len(coords_rh)):
#    print(coords_rh[i])
#    print(coords_rh_studkey[i])
#print(datacolors[coords_rh_studkey[10]])
     brain_rh.add_foci(coords_rh[i], map_surface="white", color=datacolors[coords_rh_studkey[i]], scale_factor = 0.5, hemi="rh")    
  
# brain_rh.add_foci(coords_rh_s2, map_surface="white", color="g", scale_factor = 0.5, hemi="rh")
# brain_rh.add_foci(coords_rh_s3, map_surface="white", color="r", scale_factor = 0.5, hemi="rh")
# brain_rh.add_foci(coords_rh_s5, map_surface="white", color="m", scale_factor = 0.5, hemi="rh")
# brain_rh.add_foci(coords_rh_s6, map_surface="white", color="y", scale_factor = 0.5, hemi="rh")
# brain_rh.add_foci(coords_rh_s7, map_surface="white", color="k", scale_factor = 0.5, hemi="rh")
# brain_rh.add_foci(coords_rh_s8, map_surface="white", color="#33F8FF", scale_factor = 0.5, hemi="rh")

In [219]:
brain_rh.show_view('frontal')

((59.999999999999993,
  80.0,
  433.49957275390625,
  array([ 0.        ,  0.64334869,  0.        ])),
 -106.739)

In [220]:
"""
save output
"""
#with multiple arguments, one can use a tuple as operand: 'foo %d, bar %d' % (foo, bar)
brain_rh.save_image("%s_lat_%s.png" % (subject_id, currhem))

In [221]:
###############################################################################
# # Get a set of images as a montage, note the data could be saved if desired
# #image = brain_rh.save_montage(None, ['l', 'v', 'm'], orientation='v')
# image = brain.save_montage(None, ['l', 'v', 'm'], orientation='v')
# brain.close()



In [222]:
###############################################################################
# View created image
# import pylab as pl
# fig = pl.figure(figsize=(5, 3), facecolor='white')
# ax = pl.axes(frameon=False)
# ax.imshow(image, origin='upper')
# pl.xticks(())
# pl.yticks(())
# pl.draw()
# pl.show()